# Продвинутое программирование на языке Python
## Семинар 1. Принципы ООП

На этом занятии постараемся разобрать логику `.__init__()` -> `.fit()` -> `.transform()` (`.predict()`). Это пригодится нам при работе с `sklearn`.

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [3]:
X = np.random.uniform(-10, 10, (100, 2))
y = np.random.normal(0, 1, 100)

X_new = np.random.uniform(-10, 10, (50, 2))

In [4]:
lr = LinearRegression()  # .__init__()
# Y = b1*X1 + b2*X2 + ... + bn*Xn

In [6]:
type(lr)

sklearn.linear_model._base.LinearRegression

In [7]:
dir(lr)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 '_validate_params',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_metadata_routing',
 'get_params',
 'n_jobs',
 'positive',
 'predict',
 'score',
 'set_fit_request',
 's

In [8]:
lr.fit(X, y)

LinearRegression()

In [9]:
dir(lr)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 '_validate_params',
 'coef_',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_metadata_routing',
 'get_params',
 'intercept_',
 'n_features_in_',
 'n_jobs',
 'positive',
 

In [10]:
lr.coef_

array([-0.00124705, -0.00748942])

In [11]:
predict = lr.predict(X_new)

В процессе анализа данных нам часто нужно будет трансформировать их каким-либо образом. Рассмотрим две главных трансформации: стадартизацию и нормализацию.

![st&norm](https://i.stack.imgur.com/XmyWR.png)

Наша задача - реализовать класс `Transformer`.

При этом:
- для инициализации класса необходим вид трансформации;
- на вход может подаваться последовательность из целых / нецелых чисел (возможные виды: `list`, `tuple`, `np.ndarray`, `pd.DataFrame`). Последовательности (если это не `np.ndarray` или `pd.DataFrame`) должны иметь степень вложенности 2 (список списков);
- параметры для трансформации должны сохраняться в качестве атрибутов объекта (для каждой переменной);
- должна поддерживаться логика `fit`-`transform`.

In [31]:
class Transformer:
    _valid_ttypes = {'norm', 'st'}
    _valid_data_types = [list, tuple, np.ndarray, pd.DataFrame]
    _valid_dtypes = [float, int]
    
    def __init__(self, ttype='norm'):
        if ttype not in self._valid_ttypes:
            raise ValueError(f'unknow ttype {ttype}, use only valid ttypes: {self._valid_ttypes}')

        self.ttype = ttype

    def _validate_data(self, data):
        data_type = type(data)

        # check data type
        if data_type not in self._valid_data_types:
            raise ValueError(f'unknow input type {data_type}, use only valid input types: {self._valid_data_types}')

        if data_type in [list, tuple]:
            data = np.array(data)
        elif data_type == pd.DataFrame:
            data = data.values

        # check scalar data type
        dtype = data.dtype

        if dtype not in self._valid_dtypes:
            raise ValueError(f'unknow data type {dtype}, use only valid data types: {self._valid_dtypes}')

        # check dimensions number
        if data.ndim != 2:
            raise ValueError(f'invalid dimensions number {data.ndim}, must be 2')

        return data

    def fit(self, data):
        data_valid = self._validate_data(data)

        # get and save params
        if self.ttype == 'norm':
            params = {
                'min': data_valid.min(axis=0),
                'max': data_valid.max(axis=0)
            }
        else:
            params = {
                'mean': data_valid.mean(axis=0),
                'std': data_valid.sd(axis=0)
            }

        self.params = params

    def transform(self, data):
        if not hasattr(self, 'params'):
            raise ValueError('you cannot call transform before fit!')
        
        data_valid = self._validate_data(data)

        # apply transformation
        if self.ttype == 'norm':
            data_transformed = (data_valid - self.params['min']) / (self.params['max'] -  self.params['min'])
        else:
            data_transformed = (data_valid - self.params['mean']) / self.params['std']

        return data_transformed

In [37]:
tr = Transformer('norm')

tr.fit(X)
X_transformed = tr.transform(X_new)

In [33]:
tr.params

{'min': array([-9.86888373, -9.70443963]),
 'max': array([9.87532466, 9.70713258])}

Теперь давайте попробуем немного изменить логику и реализовать родительский класс Transformer и два наследуемых: StandardScaler и NormScaler.

In [68]:
class Transformer:
    _valid_data_types = [list, tuple, np.ndarray, pd.DataFrame]
    _valid_dtypes = [float, int]
    
    def __init__(self):
        pass

    def __repr__(self):
        if hasattr(self, 'params'):
            representation = f'{self.__class__.__name__} object with params {self.params}'
        else:
            representation = f'Uninitialized {self.__class__.__name__} object'

        return representation
            
    def _validate_data(self, data):
        data_type = type(data)

        # check data type
        if data_type not in self._valid_data_types:
            raise ValueError(f'unknow input type {data_type}, use only valid input types: {self._valid_data_types}')

        if data_type in [list, tuple]:
            data = np.array(data)
        elif data_type == pd.DataFrame:
            data = data.values

        # check scalar data type
        dtype = data.dtype

        if dtype not in self._valid_dtypes:
            raise ValueError(f'unknow data type {dtype}, use only valid data types: {self._valid_dtypes}')

        # check dimensions number
        if data.ndim != 2:
            raise ValueError(f'invalid dimensions number {data.ndim}, must be 2')

        return data

    def _get_and_save_params(self, data):  # must be specified with descendant
        pass

    def _apply_params(self, data):  # must be specified with descendant
        pass

    def fit(self, data):
        data_valid = self._validate_data(data)

        # get and save params
        self._get_and_save_params(data_valid)

    def transform(self, data):
        if not hasattr(self, 'params'):
            raise ValueError('you cannot call transform before fit!')
        
        data_valid = self._validate_data(data)

        # apply transformation
        data_transformed = self._apply_params(data_valid)

        return data_transformed 

In [69]:
class StandardScaler(Transformer):
    def _get_and_save_params(self, data):  # must be specified with descendant
        params = {
            'mean': data.mean(axis=0),
            'std': data.std(axis=0)
        }

        self.params = params

    def _apply_params(self, data):  # must be specified with descendant
        data_transformed = (data - self.params['mean']) / self.params['std']

        return data_transformed

class NormScaler(Transformer):
    def _get_and_save_params(self, data):  # must be specified with descendant
        params = {
            'min': data_valid.min(axis=0),
            'max': data_valid.max(axis=0)
        }

        self.params = params

    def _apply_params(self, data):  # must be specified with descendant
        data_transformed = (data_valid - self.params['min']) / (self.params['max'] -  self.params['min'])

        return data_transformed

In [70]:
st_scaler = StandardScaler()

st_scaler.fit(X)
X_transformed = st_scaler.transform(X_new)

In [67]:
st_scaler.__class__.__name__

'StandardScaler'

In [65]:
dir(st_scaler.__class__)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply_params',
 '_get_and_save_params',
 '_valid_data_types',
 '_valid_dtypes',
 '_validate_data',
 'fit',
 'transform']

In [54]:
dir(st_scaler)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply_params',
 '_get_and_save_params',
 '_valid_data_types',
 '_valid_dtypes',
 '_validate_data',
 'fit',
 'params',
 'transform']

In [71]:
print(st_scaler)

StandardScaler object with params {'mean': array([0.99397778, 0.24381873]), 'std': array([5.8580265 , 5.99838432])}


In [45]:
issubclass(StandardScaler, Transformer)

True

In [46]:
issubclass(bool, int)

True